In [6]:
from IPython.display import HTML

In [7]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Code aan/uit schakelen."></form>''')

## Data Story (Draft: last edited 22 jun 20:28)

### Terrorisme en de Media

Deze data story volgt een bevestigende benadering. We zullen twee datasets analyseren en visualiseren om bestaande hypotheses te toetsen en overtuigende argumenten te presenteren ter ondersteuning van de verschillende perspectieven die we gedurende deze data story zullen behandelen.

Onze focus ligt op de escalatie van terrorisme in zowel westerse als niet-westerse landen en de potentiele invloed van de media. Ook zullen we licht schijnen op het feit dat de media veel van de aanslagen in het westen laat zien en weinig van het niet-westen. Deze veronderstelling wordt ondersteund door eerdere studies (https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2505359), die hebben aangetoond dat de media zich voornamelijk richt op terrorisme in het westen.

Tijdens onze analyse zullen we rigoureus de gegevens onderzoeken en opvallende patronen en trends benadrukken. We maken gebruik van gegevensvisualisaties om helder inzicht te verschaffen in de manifestatie van terrorisme en eventuele verschillen tussen westerse en niet-westerse landen. Daarnaast zullen we de focus leggen op de berichtgeving in de media en onderzoeken hoe deze mogelijk van invloed is op het publieke beeld van terrorisme.

Met deze data story streven we ernaar om een dieper inzicht te bieden in de complexiteit rondom terrorisme en de rol van de media bij het vormen van percepties. We willen aantonen dat er meerdere perspectieven bestaan en dat het essentieel is om een breed scala aan informatiebronnen te raadplegen om een volledig beeld te krijgen. Door middel van een bevestigende benadering willen we bestaande hypothesen toetsen en concrete inzichten verschaffen die kunnen bijdragen aan een meer gefundeerde discussie over terrorisme en mediaberichtgeving.

In [3]:
# Import packages
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [2]:
'''
Nb. for the sake of memory on GithubPages, we have modified the global terrorism database and filtered the columns accoording to our needs. 
This step is discarded in the code and it has been done before starting with the actual preprocessing of the data. 
So now we have only 14 columns instead of 135.
'''

# Load global terrorism and population datasets using pandas
df_1 = pd.read_csv('globalterrorismdb_0718dist.csv', encoding = 'latin-1', low_memory = False)
df_2 = pd.read_csv('Population.csv')

In [3]:
#data preprocessing
df_1['west_or_not'] = df_1['region'].apply(lambda x: '1' if x == 1 or x == 12 or x == 8 else '0')
df_1['Event'] = 1

population = df_2.sort_values(by=['country_txt'])

filtered = df_1.filter(items=['country', 'Event', 'success', 'nkill', 'nwound'])
grouped = filtered.groupby("country").sum()
grouped = grouped.sort_values(by=['country'])

filteredwest = df_1.filter(items=['country', 'country_txt', 'west_or_not'])
groupedwest = filteredwest.drop_duplicates(subset=['country'])
groupedwest = groupedwest.sort_values(by=['country'])

totalset = groupedwest.join(grouped, how="inner", on="country")
totalset = totalset.merge(population, how='inner', on='country_txt')
totalset = totalset.drop(columns=['country', 'World', '#']) 

totalset = totalset.reset_index()
totalset = totalset.drop(columns=["index"])

west = totalset[totalset['west_or_not'] != '0']
nonwest = totalset[totalset['west_or_not'] != '1']

In [4]:
def cases_count(df_1, column_name):

    cases_count = df_1.groupby(column_name)['eventid'].count()
    
    # Create a new DataFrame from the Series object
    cases_count_df = pd.DataFrame({column_name: cases_count.index, 'Aantal gevallen': cases_count.values})

    # Sort the DataFrame by the number of cases ('Aantal gevallen') column in descending order
    cases_count_df = cases_count_df.sort_values('Aantal gevallen', ascending=False)
    
    
    return cases_count_df

In [5]:
def cases_sum(df_1, col_group_by, value):

    cases_sum = df_1.groupby(col_group_by)[value].sum()
    
    # Create a new DataFrame from the Series object
    cases_sum_df = pd.DataFrame({col_group_by: cases_sum.index, value: cases_sum.values})

    # Sort the DataFrame by the number of cases ('Aantal gevallen') column in descending order
    cases_sum_df = cases_sum_df.sort_values(value, ascending=False)
    
    
    return cases_sum_df


In [6]:

# Total cases
cases_year_count_df = cases_count(df_1,"iyear")

# Extended cases (incident extended more than 24 hours)
extended_cases_sum_df = cases_sum(df_1,'iyear','extended')

# Success cases
success_cases_sum_df = cases_sum(df_1,'iyear','success')

# Suicide cases
suicide_cases_sum_df = cases_sum(df_1,'iyear','suicide')

# Merge columns on "iyear"
cases_by_year_df = pd.merge(cases_year_count_df, extended_cases_sum_df, on = 'iyear', how='left').merge(success_cases_sum_df, on = 'iyear').merge(suicide_cases_sum_df, on = 'iyear')

In [ ]:
# Create a line chart using Plotly
cases_by_year_df = cases_by_year_df.sort_values('iyear', ascending=False)

# Create Scatter traces for extended cases and success cases and suicide cases
total_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['Aantal gevallen'], mode='lines', name='Totaal gevallen')
success_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['success'], mode='lines', name='Succesvolle gevallen')
extended_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['extended'], mode='lines', name='Uitgebreide gevallen')
suicide_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['suicide'], mode='lines', name='Zelfmoordgevallen')

# Add traces to a single Figure object
fig = go.Figure(data=[total_cases, success_cases, extended_cases, suicide_cases])
fig.update_layout(title='Aantal aanslagen wereldwijd door de jaren heen', xaxis_title='Year', yaxis_title='Cases')
fig.show()


De bovenstaande grafiek toont een opvallend sterke stijgende trend in het aantal aanslagen wereldwijd door de jaren heen. Aan het moment waarop de sterke groei is ingezet zouden we kunnen afleiden dat dit te maken heeft met de media. Rond die tijd is media groter geworden naar aanleding van de opkomst van de mobiele telefoon. De vraag is echter, zijn er daadwerkelijk meer aanslagen gebeurd of is het aantal hetzelfde gebleven en weten we van meer aanslagen door het gebruik van de mobiele telefoon. 
Aanslagen die in buiten gebieden gebeurden, zoals een dorp ergens midden in een woestijn, konden wij vroeger erg gemakkelijk niets over weten. wij hebben namelijk weinig tot geen contact met deze mensen omdat het erg verweg is. Tegenwoordig kunnen de mensen uit zo'n buitengebied alles posten en gemakkelijk mensen bellen waardoor de informatie snel verspreid word.

In [8]:
cases_count_df = cases_count(df_1, "country_txt")

# create choropleth map
fig = px.choropleth(cases_count_df, 
                    locations='country_txt',
                    locationmode='country names',
                    color='Aantal gevallen',
                    hover_name='country_txt',
                    projection='natural earth')

fig.update_layout(title_text='Aantal aanslagen over de wereld')


fig.show()

In bovenstaand figuur is een wereldkaart afgebeeld met de daarbij het aantal aanslagen wat er in het desbetreffende land is gebeurd. Vanuit het figuur is het duidelijk te zien dat de landen met het hoogste aantal aanslagen zich bevinden in het Midden-Oosten en Azië, en niet in westerse landen. Dit staat haaks op de berichtgeving in de media, die de nadruk legt op aanslagen in het Westen. 


#In de bovenstaande figuur worden het aantal aanslagen wereldwijd weergegeven, waarbij het precieze aantal aanslagen #per land zichtbaar wordt wanneer de muis over het land wordt geplaatst. Vanuit de kaart is het duidelijk te zien #dat de landen met het hoogste aantal aanslagen zich bevinden in het Midden-Oosten en Azië, en niet in westerse #landen. Dit staat in contrast met de berichtgeving in de media, die juist vaak de nadruk legt op aanslagen in het #Westen.

#Op het eerste gezicht lijkt dit argument logisch, maar het is belangrijk om te realiseren dat de landen in het #Midden-Oosten en Azië enkele extreme gevallen vertegenwoordigen, en dat het niet gaat om het totale aantal #aanslagen in niet-westerse landen in vergelijking met westerse landen. Om een duidelijker beeld te schetsen van de #verdeling van aanslagen tussen westerse en niet-westerse landen, zullen we gebruikmaken van aanvullende #visualisaties.

In [9]:
fig = go.Figure()
fig.add_trace(go.Bar(name='Aantal aanslagen per land', x= west['country_txt'], y=west['Event']))
fig.add_trace(go.Bar(name='Aantal doden per land', x= west['country_txt'], y=west['nkill']))
fig.add_trace(go.Bar(name='Aantal succesvolle aanslagen per land', x= west['country_txt'], y=west['success']))

fig = fig.update_layout(
    title="Aantal (succesvolle) aanslagen en gevallen doden in westerse landen",
    autosize=True,
    annotations=[dict(xref='paper',
                    yref='paper',
                    y= -0.45,
                    showarrow=False,
                    text='Westerse landen')])

fig.show()

In bovenstaand figuur worden de westerse landen weergegeven waar de meeste aanslagen hebben plaatsgevonden, samen met het aantal aanslagen per land, het totale aantal doden en het aantal succesvolle aanslagen. Opvallend is dat in de meeste weergegeven landen het aantal aanslagen en het aantal succesvolle aanslagen dicht bij elkaar liggen. Echter, in vier gevallen (VS, Noorwegen, Mexico en Canada) is het aantal doden hoger dan het aantal succesvolle aanslagen. Dit kan mogelijk verklaren waarom de media deze specifieke succesvolle aanslagen meer belicht.

Deze discrepantie tussen het aantal doden en het aantal succesvolle aanslagen in bepaalde landen is een belangrijk aspect om te onderzoeken. Het roept vragen op over de mogelijke factoren die van invloed zijn op de media-aandacht voor aanslagen. Er kunnen verschillende redenen zijn waarom de media zich meer richt op succesvolle aanslagen, zoals de ernstige gevolgen voor het menselijk leven en de maatschappij, de impact op internationale betrekkingen of de perceptie van bedreiging binnen de samenleving.

In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=west["country_txt"].unique(), y=west["nkill"], name="Aantal doodde", mode="lines"))
fig.add_trace(go.Scatter(x=west["country_txt"].unique(), y=west["nwound"], name="Aantal gewonden", mode="lines"))

fig.add_trace(go.Scatter(x=nonwest["country_txt"].unique(), y=nonwest["nkill"], name="Aantal doodde", mode="lines", visible=False))
fig.add_trace(go.Scatter(x=nonwest["country_txt"].unique(), y=nonwest["nwound"], name="Aantal gewonden", mode="lines", visible=False))

fig = fig.update_layout(
            updatemenus=[
                dict(
                type = 'buttons',
                direction = 'left',
                buttons =list([
                    dict(
                        label = "Westerse landen",
                        method="update",
                        args=[{"visible": [True, True, False, False]},
                             {"title": "Aantal doodden en gewonden westerse landen"}]   
                    ),
                     dict(
                        label = "Niet westerse landen",
                        method="update",
                        args=[{"visible": [False, False, True, True]},
                             {"title": "Aantal doodden en gewonden niet westerse landen"}]   
                    )
                ])
                )
            ])


fig.show()

In vergelijking met het vorige figuur wordt het argument met betrekking tot het dodenaantal (en het aantal gewonden) in westerse landen weerlegd door de bovenstaande figuur. Deze visualisatie laat zien dat er in niet-westerse landen een stuk meer doden en gewonden zijn als gevolg van aanslagen in vergelijking met westerse landen. Indien de media hun berichtgeving zou baseren op het aantal doden en gewonden, zouden juist niet-westerse landen meer belicht moeten worden.

Deze observatie roept belangrijke vragen op over de manier waarop de media aanslagen selecteert en belicht. Als het criterium voor berichtgeving inderdaad het aantal slachtoffers is, zou men verwachten dat niet-westerse landen meer aandacht krijgen vanwege de hogere aantallen doden en gewonden. Het feit dat dit niet altijd het geval lijkt te zijn, suggereert dat er mogelijk andere factoren een rol spelen bij de keuze van de media om bepaalde aanslagen te benadrukken.

In [11]:
labels = nonwest['country_txt']
values = nonwest['Event']

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='none',
                            insidetextorientation='radial'
                            )])

fig.show()

De pie chart hierboven toont het aantal aanslagen in niet-westerse landen en geeft aan welk percentage elk land bijdraagt aan het totale aantal aanslagen. Deze visuele weergave benadrukt het belang van niet-westerse landen in termen van het absolute aantal aanslagen.

Deze bevinding versterkt het argument dat de media zich meer zou moeten richten op aanslagen in niet-westerse landen, gezien het grote aantal aanslagen dat daar plaatsvindt. Hoewel westerse landen weliswaar betrokken zijn bij aanslagen, is het duidelijk dat niet-westerse landen in absolute zin een significant deel van de wereldwijde aanslagen vertegenwoordigen.

Door de media-aandacht voornamelijk te richten op aanslagen in westerse landen, kan er een vertekend beeld ontstaan van het wereldwijde terrorisme. Het is essentieel om een evenwichtige en objectieve verslaggeving na te streven die recht doet aan de werkelijke verdeling van aanslagen over de wereld. Dit betekent dat er meer aandacht moet worden besteed aan niet-westerse landen, waar het aantal aanslagen vaak aanzienlijk is.

In [12]:
totalset = totalset.groupby("west_or_not").sum()
totalset['placement'] = ['Non westren country', 'Westren country']
totalset['equated'] = totalset['Event'] / totalset['Population']

In [13]:
labels = totalset['placement']
values1 = totalset['Event']
values2 = totalset['equated']

fig = go.Figure()
colors = ['red', 'green']
fig.add_trace(
    go.Pie(labels=labels, values=values1, hole=.8, textinfo='label+percent'))
fig.add_trace(
    go.Pie(labels=labels, values=values2, hole=.8, textinfo='label+percent', visible=False))

fig = fig.update_layout(
            updatemenus=[
                dict(
                type = 'buttons',
                direction = 'left',
                buttons = list([
                    dict(
                        label = "Origineel",
                        method = "update",
                        args = [{"visibale": [True, False]},
                               {"title": "Aantal aanslagen in westerse en niet westers landen"}]
                    ),
                    dict(
                        label = "Gelijkgesteld",
                        method = "update",
                        args = [{"visible": [False, True]},
                               {"title": "Aantal aanslagen in westerse en niet westerse landen (gelijkgesteld)"}]
                    )
                ])
                )
            ])

fig = fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=1)))
fig = fig.update_layout(title_text='Aantal aanslagen westerse en niet westerse landen', showlegend=False)

fig.show()

Als laatste wordt hierboven een donut chart weergegeven (oorspronkelijke versie) met het totale aantal aanslagen in de wereld en de percentages van aanslagen in westerse en niet-westerse landen. Uit deze grafiek blijkt dat bijna 90% van de aanslagen plaatsvond in niet-westerse landen. Op basis hiervan lijkt de media-aandacht voor aanslagen in westerse landen een onlogische keuze, aangezien deze landen slechts iets meer dan 10% van het totale aantal aanslagen vertegenwoordigen. Deze donut chart suggereert dat de media opzettelijk kiest om aanslagen in het westen aan het publiek te tonen.

Echter, wanneer we kijken naar de aangepaste donut chart (gebaseerd op gelijkgestelde aantallen), zien we dat de verhoudingen van aanslagen slechts 9,4% verschillen. Deze donut chart houdt rekening met het percentage aanslagen in verhouding tot de bevolking. Aangezien het aantal inwoners in niet-westerse landen aanzienlijk groter is dan in westerse landen, kan de keuze van de media om aanslagen in het westen te belichten als "logisch" worden beschouwd, omdat de verhouding tussen aanslagen en bevolking van beide regio's dicht bij elkaar ligt. Dit betekent echter niet dat het aantal doden en gewonden geen rol speelt bij de beslissing van de media om aanslagen te benadrukken; dit kan per aanslag verschillen.

Deze bevindingen werpen een interessant licht op de complexiteit van de media-aandacht voor aanslagen. Hoewel de oorspronkelijke donut chart suggereert dat de media selectief is in het belichten van aanslagen in westerse landen, laat de aangepaste versie zien dat de verhoudingen tussen aanslagen en populatie vergelijkbaar zijn. Het aantal doden en gewonden blijft echter een belangrijke factor bij het bepalen van de media-aandacht. Deze discussie benadrukt de noodzaak van een kritische benadering en een diepgaand begrip van de verschillende factoren die de media-aandacht voor aanslagen beïnvloeden.

Het is van belang om de complexiteit van deze kwestie te erkennen en te streven naar een gebalanceerde verslaggeving die recht doet aan de realiteit van terrorisme, ongeacht de geografische locatie. Het bevorderen van een genuanceerde discussie over de rol van de media bij het vormgeven van het publieke bewustzijn is van essentieel belang om een beter begrip te krijgen van de dynamiek rondom aanslagen en de gevolgen ervan op de samenleving als geheel.